# Tenencia de acciones

## Importo las librerias necesarias

In [1]:
from google.cloud import firestore
import os
import pandas as pd

Guardo como variable el acceso a la base de datos de Firestore y lo inicializo

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ='../acciones-usd-real-firebase-adminsdk-rfxvv-c888549652.json'

# Inicializo firestore
db = firestore.Client()

Hago la consulta a la base y traigo todos los tickers, junto con su PPC y la cantidad de cada uno, luego los convierto en un DataFrame

In [3]:
list_tickers = []
try:
    # Consulto los tickers y los guardo en una lista
    tickers = db.collection('acciones').stream()
    for ticker in tickers:
        list_tickers.append(ticker.to_dict())
    
    # Convierto la lista en un data frame
    df_tickers = pd.DataFrame(list_tickers)
except Exception as e:
    print(f'No fue posible consultar los datos, error: {e}')


Consulto el valor del dola CCL al día de hoy utilizando la api de mercados de ambito

In [4]:
from datetime import datetime
import requests

# Funcion para obtener el valor del dolcar CCL al dia de la fecha
def valor_ccl():
    FECHA_INICIO = datetime.now()
    FECHA_FINAL = datetime.now()
    # URL de la API
    url_ccl = f"https://mercados.ambito.com//dolarrava/cl/grafico/{FECHA_INICIO.date()}/{FECHA_FINAL.date()}"
    #url_mep = f'https://mercados.ambito.com//dolarrava/mep/grafico/{FECHA_INICIO.date()}/{FECHA_FINAL.date()}'

    # Realizar una solicitud GET
    response = requests.get(url_ccl)

    # Verificar el estado de la respuesta
    if response.status_code == 200:
        # Parsear el contenido JSON de la respuesta
        data = response.json()
        return data[1][1]
    else:
        print(f"Error: {response.status_code}")

dolar_ccl = valor_ccl()
#dolar_ccl = 1480
print(dolar_ccl)

1294.23


Se realiza la descarga de la información de los tickers, quedandonos unicamente con el precio de cierre.
Se realizan distintas operaciones agregando columnas para saber lo total invertido de cada acción tanto en USD como en ARS.

In [5]:
import yfinance as yf

# Las convierto en lista para poder descargar sus datos y los descargo con yfinance
list_tickers = df_tickers['Ticker'].to_list()
yf_t = yf.download(list_tickers, period='1d', interval='2m')['Adj Close']

# Elijo el ultimo registro
yf_t = yf_t.iloc[-1]

# Creo una columna donde agrego esos precios mapeandolos con el indice de la serie anterior ya que son el mismo
df_tickers['Cierre_ARS'] = df_tickers['Ticker'].map(yf_t).round(2)

# Agrego otra columna donde se ve la ganancia/perdida en ARS y una donde tengo el total invertido
df_tickers['Total invertido ARS'] = (df_tickers['Cierre_ARS'] * df_tickers['Cantidad']).round(2)
df_tickers['Gan/Perd ARS'] = ((df_tickers['Cierre_ARS'] * df_tickers['Cantidad']) - (df_tickers['PPC'] * df_tickers['Cantidad'])).round(2)

# Hago lo mismo pero en USD
df_tickers['Total invertido USD'] = (df_tickers['Cantidad'] * (df_tickers['Cierre_ARS'] / dolar_ccl)).round(2)
df_tickers['Gan/Perd USD'] = (df_tickers['Cantidad'] * (df_tickers['Cierre_ARS'] / dolar_ccl)).round(2) - ((df_tickers['PPC'] / dolar_ccl) * df_tickers['Cantidad']).round(2)
hoy = datetime.now()
df_tickers

[*********************100%%**********************]  11 of 11 completed


,Ticker,PPC,Cantidad,Cierre_ARS,Total invertido ARS,Gan/Perd ARS,Total invertido USD,Gan/Perd USD
0,AGRO.BA,59.70,280,55.7,15596.0,-1120.00,12.05,-0.87
1,AMZN.BA,1925.00,35,1650.0,57750.0,-9625.00,44.62,-7.44
2,COME.BA,96.80,741,232.0,171912.0,100183.20,132.83,77.41
3,GGAL.BA,3035.00,7,3580.0,25060.0,3815.00,19.36,2.94
4,GOOGL.BA,3998.94,17,3785.0,64345.0,-3636.98,49.72,-2.81
5,MELI.BA,18890.38,8,17425.0,139400.0,-11723.04,107.71,-9.06
6,META.BA,10900.00,9,25075.0,225675.0,127575.00,174.37,98.57
7,NVDA.BA,7163.88,21,6000.0,126000.0,-24441.48,97.36,-18.88
8,PAMP.BA,2528.00,14,2355.0,32970.0,-2422.00,25.47,-1.88
9,PFE.BA,9267.58,6,9950.0,59700.0,4094.52,46.13,3.17


Se realiza una descarga en excel de los datos.

In [6]:
# Guardo un excel con los datos
df_tickers.to_excel(f'informe-acciones-usd.xlsx')

# Visualización
Utilizando la libreria de plotly se realizan 2 gráficos para saber la participación de cada acción en nuestra cartera y para comparar lo total invertido de cada acción vs lo total ganado en cada acción.

In [7]:
import plotly.express as px
import plotly.graph_objs as go

# Calculo la ganancia total en USD
total_usd = (df_tickers['Gan/Perd USD'].sum()).round(2)

# Grafico la participación x accion en usd
fig_pie = px.pie(df_tickers, values='Total invertido USD', names='Ticker', title='Participación en USD por acción')
fig_pie.update_traces(textposition='outside', textinfo='percent+label')
fig_pie.add_annotation(text=f'La ganancia neta es de {total_usd} USD', x=-0.3, y=1.1, showarrow=False, font=dict(size=10, color='gray'), xref='paper', yref='paper')
fig_pie.show()

# Ganancia/perdida en dolares de cada accion
fig_bars = go.Figure(data=[
    go.Bar(name='Total invertido USD', x=df_tickers['Ticker'], y=df_tickers['Total invertido USD']),
    go.Bar(name='Gan/Perd USD', x=df_tickers['Ticker'], y=df_tickers['Gan/Perd USD'])
])
# Change the bar mode
fig_bars.update_layout(barmode='group', title='Total invertido y Ganancias en USD por accion')
fig_bars.show()